In [0]:
# Load it into a DataFrame
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True, inferSchema=True)
df.show(5)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+---------+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
# Scenario 1 :  Top 10 most populous cities
# df.orderBy(df["2014 Population estimate"].desc()).limit(10).show()
# display(df.orderBy(df["2014 Population estimate"].desc()).limit(10))
# display(df.select("City").orderBy(df["2014 Population estimate"].desc()).limit(10))
display(
    df_selected.orderBy('2014 Population estimate', ascending=False)
               .select('City', '2014 Population estimate').limit(10)
)

City,2014 Population estimate
New York[6],8491079
Los Angeles,3928864
Chicago,2722389
Houston[7],2239558
Philadelphia[8],1560297
Phoenix,1537058
San Antonio,1436697
San Diego,1381069
Dallas,1281047
San Jose,1015785


In [0]:
# Scenario 2 : Average median sales price by State
from pyspark.sql.functions import avg
# display( df_selected.groupBy('State').avg('2015 median sales price') )

df_selected.groupBy("State").agg(
    avg("2015 median sales price").alias("Avg Median Price")
).orderBy("Avg Median Price", ascending=False).show(10)

+--------------------+------------------+
|               State|  Avg Median Price|
+--------------------+------------------+
|             Hawai'i|             699.3|
|          California| 547.9714285714286|
|District of Columbia|             367.8|
|          New Jersey|             350.8|
|            Colorado|333.46666666666664|
|          Washington|266.29999999999995|
|       Massachusetts|             261.8|
|         Connecticut|250.13333333333333|
|                Utah|             243.3|
|       New Hampshire|             237.4|
+--------------------+------------------+
only showing top 10 rows



In [0]:
#filter cities with population > 1 million
df_selected.filter(df_selected["2014 Population estimate"]>1000000).select("City","2014 Population estimate").show()

+---------------+------------------------+
|           City|2014 Population estimate|
+---------------+------------------------+
|        Phoenix|                 1537058|
|    Los Angeles|                 3928864|
|      San Diego|                 1381069|
|       San Jose|                 1015785|
|        Chicago|                 2722389|
|    New York[6]|                 8491079|
|Philadelphia[8]|                 1560297|
|         Dallas|                 1281047|
|     Houston[7]|                 2239558|
|    San Antonio|                 1436697|
+---------------+------------------------+



In [0]:
# Scenario 4 : Identify cities with missing or zero price values
# Drop rows where 'price' is null and filter out rows where 'price' is 0
# df_cleaned = df_selected.dropna(subset=["2015 median sales price"]).filter(df_selected["2015 median sales price"] != 0)

df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"]!=0)).select("City","2015 median sales price").show()

# Show the result
# df_cleaned.show()


+-------------+-----------------------+
|         City|2015 median sales price|
+-------------+-----------------------+
|   Birmingham|                  162.9|
|   Huntsville|                  157.7|
|       Mobile|                  122.5|
|   Montgomery|                  129.0|
|Anchorage[19]|                   null|
|     Chandler|                   null|
|  Gilbert[20]|                   null|
|     Glendale|                   null|
|         Mesa|                   null|
|       Peoria|                   null|
|      Phoenix|                  206.1|
|   Scottsdale|                   null|
|     Surprise|                   null|
|        Tempe|                   null|
|       Tucson|                  178.1|
|  Little Rock|                  131.8|
|      Anaheim|                  685.7|
|      Antioch|                   null|
|  Bakersfield|                   null|
|     Berkeley|                   null|
+-------------+-----------------------+
only showing top 20 rows

